<a href="https://colab.research.google.com/github/p42777/IntroML2022/blob/main/Task3/ML22_T3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import pandas as pd
import seaborn as sns
from xgboost import *
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.feature_selection import RFE
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
try:
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
except:
    !pip install --upgrade shap-hypetune
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA

import warnings
warnings.simplefilter('ignore')

In [58]:
data = pd.read_csv('train_dec08_task3.csv')
train_data, test_data = train_test_split(data, test_size = 0.3, random_state=0)

X_train = train_data.iloc[:, :14]
y_train = train_data.iloc[:, 14]

X_test = test_data.iloc[:, :14]
y_test = test_data.iloc[:, 14]

In [59]:
# resampling due to imbalanced training data
# over-sampling (SMOTE) + under-sampling (Tomek Links)
smote = SMOTE()
tomekLinks = TomekLinks()
X_train, y_train = smote.fit_resample(X_train, y_train)
X_train, y_train = tomekLinks.fit_resample(X_train, y_train)

In [60]:
# feature selection
k = 5
test = SelectKBest(mutual_info_classif, k = k)
fit = test.fit(X_train, y_train)

X_train = fit.fit_transform(X_train, y_train)
X_test = fit.transform(X_test)

In [65]:
xgb = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, enable_categorical=True)
# xgb = BoostRFE(xgb, min_features_to_select=1, step=1)
xgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train)], 
            early_stopping_rounds=20) 
# xgb.fit(X_train, y_train)
predicted = xgb.predict(X_test)

print('Training Accuracy: ',xgb.score(X_train,y_train))
print('Testing Accuracy: ',xgb.score(X_test,y_test))

[0]	validation_0-merror:0.219797
Will train until validation_0-merror hasn't improved in 20 rounds.
[1]	validation_0-merror:0.215161
[2]	validation_0-merror:0.209928
[3]	validation_0-merror:0.206639
[4]	validation_0-merror:0.20948
[5]	validation_0-merror:0.206938
[6]	validation_0-merror:0.198714
[7]	validation_0-merror:0.196621
[8]	validation_0-merror:0.193929
[9]	validation_0-merror:0.195873
[10]	validation_0-merror:0.194228
[11]	validation_0-merror:0.191238
[12]	validation_0-merror:0.190042
[13]	validation_0-merror:0.188846
[14]	validation_0-merror:0.187799
[15]	validation_0-merror:0.186453
[16]	validation_0-merror:0.184958
[17]	validation_0-merror:0.184659
[18]	validation_0-merror:0.184061
[19]	validation_0-merror:0.183014
[20]	validation_0-merror:0.182416
[21]	validation_0-merror:0.18137
[22]	validation_0-merror:0.180024
[23]	validation_0-merror:0.179127
[24]	validation_0-merror:0.177632
[25]	validation_0-merror:0.177482
[26]	validation_0-merror:0.176884
[27]	validation_0-merror:0.

In [66]:
pred_data = pd.read_csv('test_dec08_task3_only_features.csv')
pred_data = fit.transform(pred_data)
pred = xgb.predict(pred_data)

In [67]:
df = pd.DataFrame({
            "Id": list(range(1,len(pred)+1)),
            "Category": pred
        })
df.to_csv('output.csv', index=False, header=True)